In [26]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

# Co-seismic stress changes and contribution to the right-lateral slip on the Yangsan Fault

## Stress components in rotated coordinate axes

For the counter-clockwise rotation of the $x$ and $y$ axes by an angle $\theta$, stress components in the rotated coordinate system $x^{\prime}$-$y^{\prime}$ are given in term of the original stress components as

\begin{align*}
\sigma_{x^{\prime}x^{\prime}} &= \sigma_{xx} \cos^{2} \theta  + 2 \sigma_{xy} \sin \theta \cos \theta + \sigma_{yy} \sin^{2} \theta \\
\sigma_{y^{\prime}y^{\prime}} &= \sigma_{xx} \sin^{2} \theta  - 2 \sigma_{xy} \sin \theta \cos \theta + \sigma_{yy} \cos^{2} \theta \\
\sigma_{x^{\prime}y^{\prime}} &= \frac{1}{2}(\sigma_{yy} - \sigma_{xx}) \sin 2\theta + \sigma_{xy} \cos 2\theta
\end{align*}

If we assume that the original $x$ and $y$ coordinate axes are aligned with the principal stresses, $\sigma_{1}$ and $\sigma_{3}$, respectively, and a fault plane is parallel to the $x^{\prime}$ axis, we can compute the Coulomb stress on the fault plane by knowing the normal stress, $\sigma_{y^{\prime}y^{\prime}}$, and the shear stress, $\sigma_{x^{\prime}y^{\prime}}$.

<img src="coulomb_stress_geometry.png" width="480px"/>


In [27]:
def get_rotated_stress(s1, s3, angle, mu=0.6):
    ''' prints y'y' and x'y' components of stress as well as the Coulomb stress
        when the given principal stresses s1 (most compressive), s3 (lest compressive)
        and the rotation angle in degrees are given.
        
        Note that the friction coefficient, mu, is 0.6 by default 
        but can be set by a user.'''
    th = angle * np.pi/180.0
    s33 = s1*np.sin(th)**2 + s3*np.cos(th)**2
    s13 = 0.5*(s3-s1)*np.sin(2.0*th)
    Sc = s13 - mu * s33
    print "s_y'y' = ", s33, " s_x'y' = ", s13, " Coulomb stress = ", Sc

## Principal stresses from the GPS principal strain

### Geometric relationships

\begin{align}
  \sigma_{1} &= (\lambda + 2\mu) \varepsilon_{1} + \lambda \varepsilon_{2} \\
  \sigma_{2} &= \lambda \varepsilon_{1} + (\lambda+2\mu) \varepsilon_{2}
\end{align}

In [28]:
# Coseismic horizontal principal strains from Baek et al. (2012)
e1 = -1e-8
e2 = 5e-8
# Elastic moduli
lamb = 2.5e10
mu = 2.25e10
# Coseismic horizontal principal stresses
s1 = (lamb+2.0*mu)*e1 + lamb*e2
s2 = lamb*e1 + (lamb+2.0*mu)*e2
print s1, s2

550.0 3250.0


The least compressive principal strain and stress ($\sigma_{3}$) have the orientation of N70E. $\sigma_{1}$ is 90 degrees apart from it as shown in the above figure. Following the conventions adopted in King et al. (BSSA, 1994), we set $x$ and $y$ coordinate axes to be parallel with $\sigma_{1}$ and $\sigma_{3}$, respectively. The Yangsan Fault (YSF) has a strike of N20E. Therefore, we need to compute $\sigma_{y^{\prime}y^{\prime}}$ and $\sigma_{x^{\prime}y^{\prime}}$ for a plane that is 140 degrees from the $x$ axis counterclockwise.

<img src="coulomb_stress_GPS.png" width="480px"/>

In [29]:
get_rotated_stress(s1,s2,140.0)

s_y'y' =  2134.42503985  s_x'y' =  -1329.49046657  Coulomb stress =  -2610.14549048


The negative sign of $\sigma_{x^{\prime}y^{\prime}}$ indicate that the shearing sense is left-lateral on the YSF. Also, the positive sing of $\sigma_{y^{\prime}y^{\prime}}$ means tension acting normal on the YSF. The Coulomb failure criterion with zero coheion does not even apply to this case: The YSF will open up rather than slip.

Based on the above considerations, we can conclude that the co-seismic stress changes does not contribute to the right-lateral slip on the YSF.

## From PyLith solution

### Stress tensor from an element containing the hypocenter of the 2016 Kyungju earthquakes

In [30]:
stress_solution_tensor = np.array([[2066.89,1486.05,-39.4323],[1486.05,-562.291,-54.1379],[-39.4323,-54.1379,-38.7095]])
print stress_solution_tensor

[[ 2066.89    1486.05     -39.4323]
 [ 1486.05    -562.291    -54.1379]
 [  -39.4323   -54.1379   -38.7095]]


### Compute principal stresses

In [31]:
# Compute
w, v = np.linalg.eig(stress_solution_tensor)
print w[0], v[:,0] # sigma_3
print w[1], v[:,1] # sigma_1
print w[2], v[:,2]

2737.57890667 [ 0.91147782  0.41081504 -0.02095688]
-1232.6819778 [ 0.41021559 -0.91156523 -0.02778496]
-39.0074288748 [ 0.03051804 -0.01672854  0.99939422]


In [32]:
# sigma_1 orientation
print np.arctan2(-0.91156523, 0.41021559)*180.0/np.pi
# sigma_3 orientation
print np.arctan2(0.41081504, 0.91147782,)*180.0/np.pi

-65.7716673566
24.2617158925


$\sigma_{1}$ has the value of $-1232.7$ Pa and is almost horionztal in the direction of N155.8$^{\circ}$E. 
$\sigma_{3}$ is also almost horizontal and is 2737.6 Pa in in the direction of N65.7$^{\circ}$E.

Then, the counterclockwise angle from $\sigma_{1}$ to the YSF is 135.8$^{\circ}$.

<img src="coulomb_stress_PyLith.png" width="480px"/>

In [33]:
get_rotated_stress(w[1],w[0],135.8)

s_y'y' =  807.876559465  s_x'y' =  -1984.35646973  Coulomb stress =  -2469.08240541


Again, the negative sign of $\sigma_{x^{\prime}y^{\prime}}$ indicate that the shearing sense is left-lateral on the YSF. Also, the positive sing of $\sigma_{y^{\prime}y^{\prime}}$ means tension acting normal on the YSF. 

Based on the above considerations, we can conclude that the co-seismic stress changes does not contribute to the right-lateral slip on the YSF.

In [34]:
theta_list = np.linspace(0.0, 180.0, 19)
for theta in theta_list:
    print "theta = ", theta, " deg:"
    get_rotated_stress(w[1],w[0],theta)

theta =  0.0  deg:
s_y'y' =  2737.57890667  s_x'y' =  0.0  Coulomb stress =  -1642.547344
theta =  10.0  deg:
s_y'y' =  2617.86089231  s_x'y' =  678.954598374  Coulomb stress =  -891.761937009
theta =  20.0  deg:
s_y'y' =  2273.14660858  s_x'y' =  1276.01725188  Coulomb stress =  -87.8707132661
theta =  30.0  deg:
s_y'y' =  1745.01368556  s_x'y' =  1719.1733928  Coulomb stress =  672.16518147
theta =  40.0  deg:
s_y'y' =  1097.16274816  s_x'y' =  1954.97185026  Coulomb stress =  1296.67420136
theta =  50.0  deg:
s_y'y' =  407.734180712  s_x'y' =  1954.97185026  Coulomb stress =  1710.33134183
theta =  60.0  deg:
s_y'y' =  -240.116756681  s_x'y' =  1719.1733928  Coulomb stress =  1863.24344681
theta =  70.0  deg:
s_y'y' =  -768.249679705  s_x'y' =  1276.01725188  Coulomb stress =  1736.9670597
theta =  80.0  deg:
s_y'y' =  -1112.96396343  s_x'y' =  678.954598374  Coulomb stress =  1346.73297643
theta =  90.0  deg:
s_y'y' =  -1232.6819778  s_x'y' =  2.43108364198e-13  Coulomb stress =  7

Based on the above calculations, we can see that the co-seismic stress changes can promote right-lateral slip on a fault plane between ~30$^{\circ}$ to ~80$^{\circ}$ from $\sigma_{1}$ (shaded range in the figure below). The orientation of the YSF is clearly not in this range.

<img src="coulomb_stress_RLSS.png" width="480px"/>